In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics import Environment
from ostk.physics.coordinate import Frame
from ostk.physics.environment.atmospheric import Earth as EarthAtmosphericModel
from ostk.physics.environment.gravitational import Earth as EarthGravitationalModel
from ostk.physics.environment.magnetic import Earth as EarthMagneticModel
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.time import DateTime, Duration, Instant, Scale, Time
from ostk.physics.units import Mass, Length, Angle

from ostk.astrodynamics.trajectory.state import NumericalSolver
from ostk.astrodynamics.trajectory import StateBuilder, Propagator
from ostk.astrodynamics.trajectory.state import CoordinatesSubset
from ostk.astrodynamics.trajectory.state.coordinates_subset import CartesianPosition, CartesianVelocity
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics import Dynamics
from ostk.astrodynamics.dynamics import Thruster
from ostk.astrodynamics.flight.system import SatelliteSystem, PropulsionSystem
from ostk.astrodynamics.guidance_law import QLaw
from ostk.astrodynamics.solvers import FiniteDifferenceSolver
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE

## User inputs

In [ ]:
earth = Earth.from_models(
    EarthGravitationalModel(EarthGravitationalModel.Type.Spherical),
    EarthMagneticModel(EarthMagneticModel.Type.Undefined),
    EarthAtmosphericModel(EarthAtmosphericModel.Type.Undefined),
)

environment = Environment(Instant.J2000(), [earth])

In [ ]:
mass = Mass.kilograms(200.0)
wet_mass = Mass.kilograms(14.0)
propulsion_system = PropulsionSystem(thrust_si_unit=14.3e-3, specific_impulse_si_unit=1016.0)

## Setup environment, initial state and Satellite System

In [ ]:
satellite_geometry = Composite(Cuboid(
    Point(0.0, 0.0, 0.0), np.eye(3).tolist(), [1.0, 0.0, 0.0]
))
satellite_system = SatelliteSystem(mass, satellite_geometry, np.eye(3), 2.0, 2.2, propulsion_system)

In [ ]:
instant = Instant.date_time(DateTime(2024,10,1,0,0,0), Scale.UTC)
orbit = Orbit.sun_synchronous(instant, Length.kilometers(585.0), Time(11, 0, 0), Earth.default())
initial_state = orbit.get_state_at(instant)
coe = COE.cartesian((initial_state.get_position(), initial_state.get_velocity()), EarthGravitationalModel.EGM2008.gravitational_parameter)

In [ ]:
state_builder = StateBuilder(
    frame=Frame.GCRF(),
    coordinates_subsets=[
        CartesianPosition.default(),
        CartesianVelocity.default(),
        CoordinatesSubset.mass()
    ],
)

coordinates = [
    *initial_state.get_coordinates().tolist(),
    mass.in_kilograms() + wet_mass.in_kilograms(),
]

propagation_state = state_builder.build(initial_state.get_instant(), coordinates)

In [ ]:
numerical_solver = NumericalSolver(NumericalSolver.LogType.NoLog, NumericalSolver.StepperType.RungeKutta4, 2.0, 1e-12, 1e-12)

## SMA targeting

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis() + Length.kilometers(5.0),
    coe.get_eccentricity(),
    coe.get_inclination(),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)

guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={COE.Element.SemiMajorAxis: (1.0, 50.0)},
        m=3,
        n=4,
        r=2,
        b=0.01,
        k=100,
        periapsis_weight=0.0,
        minimum_periapsis_radius=Length.kilometers(6578.0),
    ),
    QLaw.GradientStrategy.Analytical
)

In [ ]:
dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.hours(12.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "semi-major axis": float(blmshort.get_semi_major_axis().in_kilometers()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="semi-major axis", color="mass", title=f"Semi-Major Axis (initial, target): ({float(coe.get_semi_major_axis().in_kilometers()):.2f}, {float(target_coe.get_semi_major_axis().in_kilometers()):.2f})")
fig.show()

### Eccentricity targeting

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis(),
    coe.get_eccentricity() + 1e-3,
    coe.get_inclination(),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={COE.Element.Eccentricity: (1.0, 1e-5)},
        m=3,
        n=4,
        r=2,
        k=100,
        periapsis_weight=0.0,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    QLaw.GradientStrategy.Analytical
)

In [ ]:
dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.days(1.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "eccentricity": float(blmshort.get_eccentricity()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="eccentricity", color="mass", title=f"Eccentricity (initial, target): ({float(coe.get_eccentricity()):.3f}, {float(target_coe.get_eccentricity()):.3f})")
fig.show()

### Inclination targeting

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis(),
    coe.get_eccentricity(),
    coe.get_inclination() + Angle.degrees(5e-2),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={COE.Element.Inclination: (1.0, 1e-5)},
        m=3,
        n=4,
        r=2,
        k=100,
        periapsis_weight=0.0,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    QLaw.GradientStrategy.Analytical
)

In [ ]:
dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.days(2.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "inclination": float(blmshort.get_inclination().in_degrees()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="inclination", color="mass", title=f"Inclination (initial, target): ({float(coe.get_inclination().in_degrees()):.4f}, {float(target_coe.get_inclination().in_degrees()):.4f})")
fig.show()

### Right Ascension of Ascending Node targeting

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis(),
    coe.get_eccentricity(),
    coe.get_inclination(),
    coe.get_raan() - Angle.degrees(1e-1),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLaw.Parameters(
        element_weights={COE.Element.Raan: (1.0, 1e-6)},
        m=3,
        n=4,
        r=2,
        k=100,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    QLaw.GradientStrategy.Analytical
)

In [ ]:
dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.days(4.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "right ascension of ascending node": float(blmshort.get_raan().in_degrees()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="right ascension of ascending node", color="mass", title=f"RAAN (initial, target): ({float(coe.get_raan().in_degrees()):.4f}, {float(target_coe.get_raan().in_degrees()):.4f})")
fig.show()

## Multiple targets

### SMA + Ecc

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis() + Length.kilometers(5.0),
    coe.get_eccentricity() + 0.001,
    coe.get_inclination(),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLawParameters(
        element_weights={COE.Element.SemiMajorAxis: (1.0, 50.0), COE.Element.Eccentricity: (1.0, 1e-4)},
        m=3,
        n=4,
        r=2,
        k=100,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    FiniteDifferenceSolver.default()
)

dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.hours(5.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "semi-major axis": float(blmshort.get_semi_major_axis().in_kilometers()),
            "eccentricity": float(blmshort.get_eccentricity()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="semi-major axis", color="mass", title=f"SMA target: {target_coe.get_semi_major_axis().in_kilometers()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="eccentricity", color="mass", title=f"Eccentricity target: {target_coe.get_eccentricity()}")
fig.show()

### Semi Major Axis + Inclination

In [ ]:
# Increasing the COE target SMA as it's osculating
target_coe = COE(
    coe.get_semi_major_axis() + Length.kilometers(5.0),
    coe.get_eccentricity(),
    coe.get_inclination() + Angle.degrees(1e-1),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLawParameters(
        element_weights={COE.Element.SemiMajorAxis: (1.0, 50.0), COE.Element.Inclination: (1.0, 1e-5)},
        m=3,
        n=4,
        r=2,
        k=100,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    FiniteDifferenceSolver.default()
)

dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.hours(10.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "semi-major axis": float(blmshort.get_semi_major_axis().in_kilometers()),
            "inclination": float(blmshort.get_inclination().in_degrees()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="semi-major axis", color="mass", title=f"SMA target: {target_coe.get_semi_major_axis().in_kilometers()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="inclination", color="mass", title=f"Inclination target: {target_coe.get_inclination().in_degrees()}")
fig.show()

### SMA + Ecc + Inclination

In [ ]:
target_coe = COE(
    coe.get_semi_major_axis() + Length.kilometers(5.0),
    coe.get_eccentricity() + 0.001,
    coe.get_inclination() + Angle.degrees(1e-1),
    coe.get_raan(),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLawParameters(
        element_weights={COE.Element.SemiMajorAxis: (1.0, 50.0), COE.Element.Eccentricity: (1.0, 1e-5), COE.Element.Inclination: (1.0, 1e-5)},
        m=3,
        n=4,
        r=2,
        k=100,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    FiniteDifferenceSolver.default(),
)

dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.hours(10.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "semi-major axis": float(blmshort.get_semi_major_axis().in_kilometers()),
            "eccentricity": float(blmshort.get_eccentricity()),
            "inclination": float(blmshort.get_inclination().in_degrees()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="semi-major axis", color="mass", title=f"SMA target: {target_coe.get_semi_major_axis().in_kilometers()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="eccentricity", color="mass", title=f"Eccentricity target: {target_coe.get_eccentricity()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="inclination", color="mass", title=f"Inclination target: {target_coe.get_inclination().in_degrees()}")
fig.show()

### SMA + Ecc + Inc + Raan targeting

In [ ]:
target_coe = COE(
    coe.get_semi_major_axis() + Length.kilometers(5.0),
    coe.get_eccentricity() + 0.001,
    coe.get_inclination() + Angle.degrees(1e-1),
    coe.get_raan() - Angle.degrees(1.0),
    coe.get_aop(),
    coe.get_true_anomaly()
)
guidance_law = QLaw(
    target_coe,
    EarthGravitationalModel.EGM2008.gravitational_parameter,
    QLawParameters(
        element_weights={
            COE.Element.SemiMajorAxis: (1.0, 50.0),
            COE.Element.Eccentricity: (1.0, 1e-5),
            COE.Element.Inclination: (1.0, 1e-5),
            COE.Element.Raan: (1.0, 1e-5)
        },
        m=3,
        n=4,
        r=2,
        k=100,
        minimum_periapsis_radius=Length.kilometers(6575.0),
        b=0.01,
    ),
    FiniteDifferenceSolver.default()
)

dynamics = Dynamics.from_environment(environment) + [
    Thruster(
        satellite_system=satellite_system,
        guidance_law=guidance_law,
    )
]
propagator = Propagator(numerical_solver, dynamics)

In [ ]:
end_state = propagator.calculate_state_at(propagation_state, propagation_state.get_instant() + Duration.hours(20.0))

In [ ]:
data = []
for state in propagator.access_numerical_solver().get_observed_states():
    blmshort = COE.cartesian((state.get_position(), state.get_velocity()), earth.get_gravitational_parameter())
    data.append(
        {
            "mass": float(state.get_coordinates()[6]),
            "semi-major axis": float(blmshort.get_semi_major_axis().in_kilometers()),
            "eccentricity": float(blmshort.get_eccentricity()),
            "inclination": float(blmshort.get_inclination().in_degrees()),
            "right ascension of ascending node": float(blmshort.get_raan().in_degrees()),
            "time": state.get_instant().get_date_time(Scale.UTC)
        }
    )

df = pd.DataFrame(data)

In [ ]:
fig = px.scatter(df, x="time", y="semi-major axis", color="mass", title=f"SMA target: {target_coe.get_semi_major_axis().in_kilometers()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="eccentricity", color="mass", title=f"Eccentricity target: {target_coe.get_eccentricity()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="inclination", color="mass", title=f"Inclination target: {target_coe.get_inclination().in_degrees()}")
fig.show()

In [ ]:
fig = px.scatter(df, x="time", y="right ascension of ascending node", color="mass", title=f"RAAN target: {target_coe.get_raan().in_degrees()}")
fig.show()